In [73]:
# Shrodinger's Qat game

import pygame
import random


from qiskit import QuantumCircuit, assemble, Aer
from qiskit.visualization import plot_histogram
from qiskit_textbook.widgets import binary_widget


# Initialization
pygame.init()


# 초기 정보
score = 0
eat_gate = []  # 먹은 게이트
gate_s = ""

#FPS
clock = pygame.time.Clock()

#화면 크기 설정
screenWidth = 1000 # 가로크기
screenHeight = 700 # 세로크기
screen = pygame.display.set_mode((screenWidth,screenHeight))  #가로, 세로


# 배경 이미지
background = pygame.image.load("background.png")
background = pygame.transform.scale(background, (screenWidth, screenHeight))



# 캐릭터
cat_w = 100
cat_h = 100
character = pygame.image.load("cat.png")
character = pygame.transform.scale(character, (cat_w, cat_h))
characterWidth, characterHeight = character.get_rect().size  # img 크기 불러옴



# 캐릭터 위치
characterXpos = (screenWidth / 2) - (characterWidth / 2)
characterYpos = screenHeight - characterHeight


# 이동할 좌표
toX, toY = 0,0

# 이동속도
characterSpeed = 1

# 난수 생성: 똥 생성용
randomNumber = 30
poSpeed = 10




# 다양한 게이트들
gate_w = 150
gate_h = 150

H = pygame.image.load("H.png")
H = pygame.transform.scale(H, (gate_w, gate_h))

X = pygame.image.load("X.png")
X = pygame.transform.scale(X, (gate_w, gate_h))

M = pygame.image.load("M.png")
M = pygame.transform.scale(M, (gate_w, gate_h))

gate = [["H", H], ["X",X], ["M", M]]




# 초기 적 설정

gate_n = random.randrange(0, len(gate))
enemy = gate[gate_n][1]
gate_kind = gate[gate_n][0]

enemyWidth = gate_w
enemyHeight = gate_h





# 처음 적이 떨어지는 위치
enemyXpos = random.randrange(0, screenWidth-enemyWidth)
enemyYpos = 0




# 타이틀
pygame.display.set_caption("Shrodinger's Qat")

# 폰트 정의
game_font = pygame.font.Font(None,40) #폰트, 크기

# 전체 플레이 시간
totalTime = 0
startTicks = pygame.time.get_ticks()






def graphic_update():
    # (조정된 값으로) 전체 그래픽 출력
    screen.fill((0,0,255))
    screen.blit(background, (0,0)) 
    screen.blit(character, (characterXpos , characterYpos))
    screen.blit(enemy, (enemyXpos , enemyYpos))
    screen.blit(timer, (10,10))
    screen.blit(score_s, (10,30))
    screen.blit(eat_gate_s, (10, 50))
    
    #화면 새로고침
    pygame.display.update()     





# Event
running = True
while running:  #실행창
    
    
    
    
    
    dt = clock.tick(20)
    #print("fps: " + str(clock.get_fps()))
    screen.fill((0,0,0))
    
    
    
    
    
    
    # 어떤 이벤트가 발생했는지에 따라 좌표의 변경 정도 결정
    for event in pygame.event.get():

        # 창을 닫으면
        if event.type == pygame.QUIT:
            running = False
        
        # 키를 누른 순간에
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT: # 왼쪽 화살표 키
                toX -= characterSpeed
            if event.key == pygame.K_RIGHT: # 오른쪽 화살표 키
                toX += characterSpeed
        
        # 키를 떼는 순간에
        if event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT or event.key == pygame.K_RIGHT: # 왼쪽 화살표 키
                toX = 0
            elif event.key == pygame.K_UP or event.key == pygame.K_DOWN: # 오른쪽 화살표 키
                toY = 0
    
    # 캐릭터 좌표를 업데이트
    characterXpos += toX * dt
    characterYpos += toY * dt
    
    # 캐릭터가 화면 밖으로 빠져나가지 않게 조정
    if characterXpos < 0:
        characterXpos = 0
    elif characterXpos > screenWidth - characterWidth:
        characterXpos = screenWidth - characterWidth

    if characterYpos < 0:
        characterYpos = 0
    elif characterYpos > screenHeight - characterHeight:
        characterYpos = screenHeight - characterHeight
    
    
    
    

    
    
    # 충돌 판정 (게이트를 먹거나, Measure를 먹거나)
    characterRect = character.get_rect() # 캐릭터 판정 위치
    characterRect.left = characterXpos
    characterRect.top = characterYpos
    
    enemyRect = enemy.get_rect() # 적 판정 위치
    enemyRect.left = enemyXpos
    enemyRect.top = enemyYpos
    
    
    
    if characterRect.colliderect(enemyRect):
        
        # Measure를 먹었을 때
        if gate_kind == "M":
            print("Collision")
            running = False
        # 다른 게이트를 먹었을 때
        else:
            
            # 먹은 게이트를 표시하도록
            eat_gate.append(gate_kind)
            gate_s = gate_s +" " + gate_kind
            
            # 게이트가 다시 위에서 떨어지도록 
            randomNumberx = random.randrange(0, screenWidth-enemyWidth)
            enemyYpos = 0
            enemyXpos = randomNumberx
            score += 1
            poSpeed += 2
        
        
            # 떨어지는 게이트 종류 바꾸기
            gate_n = random.randrange(0, len(gate))
            enemy = gate[gate_n][1]
            gate_kind = gate[gate_n][0]
            
            
            graphic_update()
            
    
    
    # 똥이 다 떨어지고 나면, 똥의 세로 위치 업데이트
    randomNumberx = random.randrange(0, screenWidth-enemyWidth)
    
    if enemyYpos > screenHeight - enemyHeight:
        enemyYpos = 0
        enemyXpos = randomNumberx
        score += 1
        poSpeed += 2
        
        # 떨어지는 게이트 종류 바꾸기
        gate_n = random.randrange(0, len(gate))
        enemy = gate[gate_n][1]
        gate_kind = gate[gate_n][0]
        
    enemyYpos += poSpeed
    
    
    #타이머 업데이트
    elapsedTime = (pygame.time.get_ticks()) / 1000 # 경과시간이 ms 이므로 초단위로 표시
    timer = game_font.render("time : %s sec"%str(int(totalTime + elapsedTime)), True, (0,0,0)) #타이머 표시
    
    # 점수 표시, 색상
    score_s = game_font.render("score : %s"%str(score), True, (0,0,0))
    
    # 먹은 게이트 표시
    eat_gate_s = game_font.render("You get : %s"%str(gate_s), True, (0,0,0))
    
    
    graphic_update()

pygame.quit() #pygame 종료

Collision
